In [1]:
import numpy as np
import random
import networkx as nx
from networkx import to_dict_of_dicts
from aqcore.transformator.problems.problem import Problem
from aqsolvers import run_solver,run_solver_repeated,get_solver_info

{'0': {'2': {'weight': 8.0}}, '1': {'2': {'weight': 5.0}}, '2': {'0': {'weight': 8.0}, '1': {'weight': 5.0}}}


In [2]:
#creates example graph with one arbitrage cycle with len cycle_len
def create_graph(num_nodes, cycle_len):
    G = nx.DiGraph()
    
    # Add nodes
    for i in range(num_nodes):
        G.add_node(i)
        

        for i in range(num_nodes):
            for j in range(num_nodes):
                if i < j and j-1==i and j <= cycle_len:
                    G.add_edge(i, j, weight=1)
                elif i == cycle_len and j == 0:
                   
                    G.add_edge(i, j, weight=1)
                elif i != j:
                    weight= round(random.uniform(.99,.9999),4)
                    G.add_edge(i, j, weight=weight)

    current_val= calculate_path_value(G,cycle_len)
    
    while current_val <1.01:
        G = incement_path_value(G,cycle_len)
        current_val= calculate_path_value(G,cycle_len)
    
        

    #for i in range(cycle_len):
        
    return G

def calculate_path_value(G, highest_node):
    path_value = 1
    current_node = 0

    # Traverse from node 0 to the highest node
    for next_node in range(1, highest_node + 1):
        weight = G[current_node][next_node]['weight']
        path_value *= weight
        current_node = next_node

    # Traverse back to node 0
    weight = G[current_node][0]['weight']
    path_value *= weight

    return path_value

def incement_path_value(G, highest_node):
    path_value = 1
    current_node = 0

    # Traverse from node 0 to the highest node
    for next_node in range(1, highest_node + 1):
        G[current_node][next_node]['weight']= G[current_node][next_node]['weight']+ random.uniform(.001,.0001)
        
        current_node = next_node

    # Traverse back to node 0
    

    return G

def get_result_list(G):
    binary_list = []

    # Iterate through each key and its nested dictionary
    for outer_key, nested_dict in G.items():
        for inner_key, attributes in nested_dict.items():
            # Check if the weight is greater than or equal to 1
            binary_list.append(1 if attributes['weight'] >= 1 else 0)
    return binary_list

In [3]:
def gen_problem_list(cycle_len,lower=3, upper=10,sample_size=10):
    
    if cycle_len <3 or lower <3:
        cylcle_len = 3
        lower=3
    problems=[]

    for i in range(lower,upper):
        for j in range(sample_size):
            if cycle_len>i:
                new_cycle_len=i
            else:
                new_cycle_len=cycle_len
            x=create_graph(i,new_cycle_len-1)
            problems.append(x)

    print(problems)
    return problems
        
    

In [4]:
import itertools
from typing import Dict, List, Literal, Optional, Tuple, Any
from pydantic import Field
import numpy as np
import math
import networkx as nx
from numpy.typing import NDArray
from random import randrange
import random

import matplotlib.pyplot as plt


class ArbitrageEdgeBased(Problem):
    """
    An edge based solution to find the optimal arbitrage in a directed and complete graph
    Creates a QUBO with the size number of edges X number of edges which produces a solution vector
    where every binary position maps to an edge
    """

    name: Literal["AEB"] = "AEB"
    graph: Dict[str, Dict[str, Dict[str, float]]] = Field(name="graph")
    penalty_1: Optional[float] = None

   

    @classmethod
    def gen_decision_problems(
        cls,
        n_problems: int,
        size: Tuple[int, int] = (10,),
        seed: int = None,
        path_lenght: int = None,
        **kwargs
    ):
        """
        Description
        ----------
        Creates a random Graph where one random path has an arbitrage opportunity

        Parameters
        ----------
        n_problems
        size
        seed
        path
        kwargs

        Returns
        -------

        """
        problems = []
        rnd = random.Random(seed)
        for _ in range(n_problems):
            path = list(
                range(rnd.randint(2, size[0])) if path_lenght is None else range(path)
            )
            path.append(path[0])
            g = nx.complete_graph(size[0], create_using=nx.DiGraph)
            for (u, v) in g.edges():
                g.edges[u, v]["weight"] = rnd.uniform(0, 1)

            for i, j in enumerate(path):
                if i < len(path) - 1:
                    g.edges[j, path[i + 1]]["weight"] = rnd.uniform(i, i + 1)

            problems.append(ArbitrageEdgeBased(graph=nx.to_dict_of_dicts(g)))

        return problems

    def gen_qubo_matrix(self) -> NDArray:
        graph = nx.DiGraph(self.graph)
        num_of_edges = graph.number_of_edges()
        edge_weights = [
            graph.get_edge_data(edge[0], edge[1]).get("weight") for edge in graph.edges
        ]
        penalty_1 = (
            self.penalty_1
            if self.penalty_1 is not None
            else math.ceil(max(edge_weights))
        )

        x = {(i, j): h for h, (i, j) in enumerate(graph.edges)}

        Q = np.zeros((num_of_edges, num_of_edges))
        H_con_1 = np.zeros((num_of_edges, num_of_edges))
        H_con_2 = np.zeros((num_of_edges, num_of_edges))
        H_con_3 = np.zeros((num_of_edges, num_of_edges))

        # Constraint 1
        for i, edge in enumerate(graph.edges(data="weight")):
            H_con_1[i][i] = math.log(edge[2])

        # Constraint 2
        for node in graph.nodes():
            out_edges = {(i, j): penalty_1 for (i, j) in graph.out_edges(node)}
            in_edges = {(j, i): -penalty_1 for (j, i) in graph.in_edges(node)}
            in_out = {**out_edges, **in_edges}

            for i, edge in enumerate(in_out):
                for j, edge1 in enumerate(in_out, start=i):
                    if edge == edge1:
                        H_con_2[x[edge1]][x[edge]] -= abs(in_out[edge])
                    elif in_out[edge] != in_out[edge1]:
                        H_con_2[x[edge1]][x[edge]] += abs(in_out[edge])
                    else:
                        H_con_2[x[edge1]][x[edge]] -= abs(in_out[edge])

        # Constraint 3
        for node in graph.nodes():
            out_edges = {(i, j): penalty_1 for (i, j) in graph.out_edges(node)}

            for i, edge in enumerate(out_edges):
                H_con_3[x[edge]][x[edge]] += abs(out_edges[edge])
                for j, edge1 in enumerate(out_edges, start=i):
                    H_con_3[x[edge1]][x[edge]] -= abs(out_edges[edge])

        # adding all constraints to Q
        Q = H_con_1 + H_con_2 + H_con_3
        # make it symmetrical
        Q = (Q + Q.T) / 2
        # negate to create min problem

        return np.negative(Q)

    def get_arbitrage(self, result):
        # returns the arbitrage percentage with 1.0 == 100% from the result bit-vector as input
        graph = nx.DiGraph(self.graph)
        x = {h: (i, j) for h, (i, j) in enumerate(graph.edges)}
        edge_weights = {
            edge: graph.get_edge_data(edge[0], edge[1]).get("weight")
            for edge in graph.edges
        }
        ret = 1
        for i, n in enumerate(result):
            if n == 1:
                ret *= edge_weights[x[i]]

        return ret - 1

    def plot_solution(self, result):
        # plots the graph with the solution path marked. Takes the solution bit array as input

        if self.get_arbitrage(result) <= 0:
            print("No Arbitrage.")
            return
        graph = nx.DiGraph(self.graph)
        pos = nx.spring_layout(graph)
        nx.draw_networkx(
            graph,
            pos=pos,
            with_labels=True,
            edgelist=[
                (edge[0], edge[1])
                for edge, taken in zip(graph.edges, result)
                if taken == 1
            ],
            edge_color="red",
        )
        nx.draw_networkx_edge_labels(
            graph,
            pos=pos,
            edge_labels={
                (edge[0], edge[1]): edge[2]
                for edge, taken in zip(graph.edges.data("weight"), result)
                if taken == 1
            },
        )
        nx.draw_networkx_edges(
            graph,
            pos,
            edgelist=[
                (edge[0], edge[1])
                for edge, taken in zip(graph.edges, result)
                if taken == 0
            ],
            connectionstyle="arc3,rad=0.2",
        )

        plt.show()




In [5]:
class ArbitrageNodeBased(Problem):
    """
    A node based solution to find the optimal arbitrage in a directed and complete graph
    Creates a QUBO with the size (number of nodes * cycle length) X (number of nodes * cycle lenth)
    which produces a solution vector where every binary position maps to a node and its position in the cycle
    """

    name: Literal["ANB"] = "ANB"
    graph: Dict[str, Dict[str, Dict[str, float]]] = Field(name="graph")
    penalty_1: Optional[float] = None
    K: Optional[int] = None

   

    def gen_qubo_matrix(self) -> NDArray:
        graph = nx.DiGraph(self.graph)
        num_of_nodes = graph.number_of_nodes()
        nodes = graph.nodes
        edges = graph.edges

        edge_weights = {
            edge: graph.get_edge_data(edge[0], edge[1]).get("weight")
            for edge in graph.edges
        }
        weights = [
            graph.get_edge_data(edge[0], edge[1]).get("weight") for edge in graph.edges
        ]
        penalty_1 = (
            -self.penalty_1 if self.penalty_1 is not None else -math.ceil(max(weights))
        )
        K = self.K if self.K is not None else randrange(num_of_nodes)
        # TODO: maybe if no cycel lenth is given we should try to cycel from 1 -> num_of_nodes and compare the arbitrage to find the best

        x = {
            (node, position): position + i * K
            for i, node in enumerate(nodes)
            for position in range(K)
        }

        Q = np.zeros((num_of_nodes * K, num_of_nodes * K))

        # constraint 1
        H_con_1 = np.zeros((num_of_nodes * K, num_of_nodes * K))

        for i in nodes:

            for j in nodes:
                if i != j:
                    for k in range(K):
                        if k < K - 1:
                            H_con_1[x[(i, k)]][x[(j, k + 1)]] = math.log(
                                edge_weights[(i, j)]
                            )
                        else:
                            H_con_1[x[(i, k)]][x[(j, 0)]] = math.log(
                                edge_weights[(i, j)]
                            )

        # constraint 2
        H_con_2 = np.zeros((num_of_nodes * K, num_of_nodes * K))
        for position in range(K):
            for node in nodes:
                for node_1 in nodes:
                    if node == node_1:

                        H_con_2[x[(node, position)]][x[(node_1, position)]] -= penalty_1

                    else:
                        H_con_2[x[(node, position)]][x[(node_1, position)]] += penalty_1
                        # maybe we need to add pen 1 to everything because of - 1 * - 1 * - A

        # constraint 3
        H_con_3 = np.zeros((num_of_nodes * K, num_of_nodes * K))
        for position in range(K):
            for node in nodes:
                for node_1 in nodes:
                    if list(edges).count((node, node_1)) == 0:

                        if position < K - 1:
                            H_con_3[x[(node, position)]][
                                x[(node_1, position + 1)]
                            ] += penalty_1
                        else:
                            H_con_3[x[(node, position)]][x[(node_1, 0)]] += penalty_1

        Q = H_con_1 + H_con_2 + H_con_3

        Q = (Q + Q.T) / 2
        # negate to create min problem
        return np.negative(Q)

    def get_arbitrage(self, result):
        # returns the arbitrage percentage with 1.0 == 100% from the result bit-vector as input
        graph = nx.DiGraph(self.graph)
        x = {
            h: (node, position)
            for h, (node, position) in enumerate(
                (
                    itertools.product(
                        range(graph.number_of_nodes()),
                        range(int(len(result) / graph.number_of_nodes())),
                    )
                )
            )
        }
        edge_weights = {
            edge: graph.get_edge_data(edge[0], edge[1]).get("weight")
            for edge in graph.edges
        }
        ret = 1

        o = sorted([x[i] for i, n in enumerate(result) if n == 1], key=lambda y: y[1])
        print(o)
        for i, n in enumerate(o):
            if n[1] < len(o) - 1:
                ret *= edge_weights[((o[i])[0], (o[i + 1])[0])]
            else:
                ret *= edge_weights[((o[i])[0], (o[0])[0])]

        return ret - 1

    def plot_solution(self, result):
        # plots the graph with the solution path marked. Takes the solution bit array as input

        if self.get_arbitrage(result) <= 0:
            print("No Arbitrage.")
            return
        graph = nx.DiGraph(self.graph)
        pos = nx.nx_pydot.graphviz_layout(graph)
        nx.draw_networkx(
            graph,
            pos=pos,
            with_labels=True,
            edgelist=[
                (edge[0], edge[1])
                for edge, taken in zip(graph.edges, result)
                if taken == 1
            ],
            edge_color="red",
        )
        nx.draw_networkx_edge_labels(
            graph,
            pos=pos,
            edge_labels={
                (edge[0], edge[1]): edge[2]
                for edge, taken in zip(graph.edges.data("weight"), result)
                if taken == 1
            },
        )
        nx.draw_networkx_edges(
            graph,
            pos,
            edgelist=[
                (edge[0], edge[1])
                for edge, taken in zip(graph.edges, result)
                if taken == 0
            ],
            connectionstyle="arc3,rad=0.2",
        )

        plt.show()


In [6]:
def count_result(result):
    samples = result.samples
    sublist_counts = {}
    
    for sublist in samples:
    
        # Convert sublist to a tuple so it can be used as a key in the dictionary
    
        tuple_sublist = tuple(sublist)
        if tuple_sublist in sublist_counts:
            sublist_counts[tuple_sublist] += 1
        else:
            sublist_counts[tuple_sublist] = 1
    
    
    
    # Find the most common sublist
    
    most_common_sublist, count = max(sublist_counts.items(), key=lambda x: x[1])   
    most_common_sublist = list(most_common_sublist)
    
    return most_common_sublist

In [7]:
import uuid
import csv
import time


def experiment(cycle_len,lower,upper,sample_size,solver,method,num_sweeps):
    # Generate a random UUID
    unique_id = uuid.uuid4()
    

    result_dict=[]
    #create problem list
    problem_list=gen_problem_list(cycle_len,lower, upper,sample_size)
    print("problem list finished")
    for i,p in enumerate(problem_list):
        if method == "AEB":
            problem = ArbitrageEdgeBased(graph=to_dict_of_dicts(p))
        else:
            problem = ArbitrageNodeBased(graph=to_dict_of_dicts(p),K=3)
        qubo = problem.gen_qubo_matrix()
        want_result =get_result_list(to_dict_of_dicts(p))
        
        if solver == "SAGA":
            result = run_solver(solver,qubo,num_sweeps=num_sweeps,rec_rate=20, max_iter=50)
            correct = count_result(result)==want_result
        
        elif solver == "QAOA":
            result = run_solver(solver,qubo,num_sweeps=num_sweeps)
            correct = count_result(result)==want_result
        elif solver == "QQ":
            result = run_solver(solver,qubo, decomposer_size = 2,qpu_config={"token" :"DEV-a623b107205b94959819a8de3b25edaeadd8c146"})
            print(result)
            correct = count_result(result)==want_result
        elif solver == "DQ":
            result = run_solver(solver,qubo,num_sweeps=100,qpu_config={"token" :"DEV-a623b107205b94959819a8de3b25edaeadd8c146"})
            print(result)
            correct = count_result(result)==want_result
            time.sleep(10)
            
        else:
            result = run_solver_repeated(solver,qubo,1000,True,num_sweeps=num_sweeps)

            correct = result[0][0]==want_result
        result_dict.append([i,solver,result.energies,result.params,method,qubo[0],qubo.shape[0],want_result,count_result(result),correct,result.runtime])
        print(i)
        
    



    # Your column headers
    headers = ['Run Number', 'Solver', 'Energie', 'Params', 'Method', 'Qubo','Dimensions', 'Want Result', 'Result', 'Correct', 'Solve Time']
    
   
    
    # Specify the file name
    file_name = str(unique_id)+'.csv'
    
    # Writing to the CSV file
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
    
        # Write the headers
        writer.writerow(headers)
    
        # Write the data
        for row in result_dict:
            writer.writerow(row)
            
        
        
    

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots


def plot_experiment(files):
    #sns.set_theme(style="whitegrid")

    # Create a figure
    
    plt.figure(figsize=(10, 6))
    plt.style.use(['science','ieee'])
    sns.set_palette("tab10")
    for file in files:
        df = pd.read_csv(file)
        df['group'] = df.index // 6
        true_counts = df.groupby('group')['Correct'].sum().reset_index()
        true_counts.columns = ['Nodes', 'True Count']
        true_counts["Nodes"] = true_counts["Nodes"] + 3
        true_counts["Percentage"] = (true_counts["True Count"] / 6)

        # Plot each file's data
        sns.lineplot(x='Nodes', y='Percentage', data=true_counts, label=df["Solver"][0])

    # Adding title and labels
    plt.title('Success Rate by QUBO Problem Size Across Multiple Experiments')
    plt.xlabel('Number of Nodes')
    plt.ylabel('Accuracy')

    # Show legend
    plt.legend()

    # Show the plot
    plt.show()


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_experiment_std(files):
    plt.figure(figsize=(10, 6))
    plt.style.use(['science','ieee'])
    sns.set_palette("tab10")

    all_data = []

    for file in files:
        df = pd.read_csv(file)
        df['group'] = df.index // 50
        true_counts = df.groupby('group')['Correct'].sum().reset_index()
        true_counts.columns = ['Nodes', 'True Count']
        true_counts["Nodes"] = true_counts["Nodes"] + 3
        true_counts["Percentage"] = (true_counts["True Count"] / 50)

        all_data.append(true_counts)
        #sns.lineplot(x='Nodes', y='Percentage', data=true_counts, label=df["Solver"][0])

    # Combine all data for mean and std calculation
    combined_data = pd.concat(all_data)
    mean_data = combined_data.groupby('Nodes')['Percentage'].mean().reset_index()
    std_data = combined_data.groupby('Nodes')['Percentage'].std().reset_index()

    # Plotting mean and standard deviation
    sns.lineplot(x='Nodes', y='Percentage', data=mean_data, label=df["Solver"][0])

    plt.fill_between(mean_data['Nodes'], 
                     mean_data['Percentage'] - std_data['Percentage'], 
                     mean_data['Percentage'] + std_data['Percentage'], 
                     color='gray', alpha=0.2, label='Standard Deviation')

    plt.title('Success Rate by QUBO Problem Size Across Multiple Experiments')
    plt.xlabel('Number of Nodes')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scienceplots

def plot_experiment_std(filesSA, filesSAGA,filesQAOA,filesDwave):
    plt.figure(figsize=(8, 4))
    plt.style.use(['science','ieee'])
    sns.set_palette("tab10")

    all_data = []

    for file in filesSA:
        df = pd.read_csv(file)
        df['group'] = df.index // 50
        true_counts = df.groupby('group')['Correct'].sum().reset_index()
        true_counts.columns = ['Nodes', 'True Count']
        true_counts["Nodes"] = true_counts["Nodes"] + 3
        true_counts["Percentage"] = (true_counts["True Count"] / 50)

        all_data.append(true_counts)
        #sns.lineplot(x='Nodes', y='Percentage', data=true_counts, label=df["Solver"][0])

    # Combine all data for mean and std calculation
    combined_data = pd.concat(all_data)
    mean_data = combined_data.groupby('Nodes')['Percentage'].mean().reset_index()
    std_data = combined_data.groupby('Nodes')['Percentage'].std().reset_index()

    # Plotting mean and standard deviation
    sns.lineplot(x='Nodes', y='Percentage', data=mean_data, label=df["Solver"][0])

    plt.fill_between(mean_data['Nodes'], 
                     mean_data['Percentage'] - std_data['Percentage'], 
                     mean_data['Percentage'] + std_data['Percentage'], 
                      alpha=0.2, label='Standard Deviation')
    all_data = []

    for file in filesSAGA:
        df = pd.read_csv(file)
        df['group'] = df.index // 50
        true_counts = df.groupby('group')['Correct'].sum().reset_index()
        true_counts.columns = ['Nodes', 'True Count']
        true_counts["Nodes"] = true_counts["Nodes"] + 3
        true_counts["Percentage"] = (true_counts["True Count"] / 50)

        all_data.append(true_counts)
        #sns.lineplot(x='Nodes', y='Percentage', data=true_counts, label=df["Solver"][0])

    # Combine all data for mean and std calculation
    combined_data = pd.concat(all_data)
    mean_data = combined_data.groupby('Nodes')['Percentage'].mean().reset_index()
    std_data = combined_data.groupby('Nodes')['Percentage'].std().reset_index()

    # Plotting mean and standard deviation
    sns.lineplot(x='Nodes', y='Percentage', data=mean_data, label=df["Solver"][0])

    plt.fill_between(mean_data['Nodes'], 
                     mean_data['Percentage'] - std_data['Percentage'], 
                     mean_data['Percentage'] + std_data['Percentage'], 
                      alpha=0.2, label='Standard Deviation')
    all_data = []

    for file in filesQAOA:
        df = pd.read_csv(file)
        df['group'] = df.index // 10
        true_counts = df.groupby('group')['Correct'].sum().reset_index()
        true_counts.columns = ['Nodes', 'True Count']
        true_counts["Nodes"] = true_counts["Nodes"] + 3
        true_counts["Percentage"] = (true_counts["True Count"] / 10)

        all_data.append(true_counts)
        #sns.lineplot(x='Nodes', y='Percentage', data=true_counts, label=df["Solver"][0])

    # Combine all data for mean and std calculation
    combined_data = pd.concat(all_data)
    mean_data = combined_data.groupby('Nodes')['Percentage'].mean().reset_index()
    std_data = combined_data.groupby('Nodes')['Percentage'].std().reset_index()

    # Plotting mean and standard deviation
    sns.lineplot(x='Nodes', y='Percentage', data=mean_data, label=df["Solver"][0])

    plt.fill_between(mean_data['Nodes'], 
                     mean_data['Percentage'] - std_data['Percentage'], 
                     mean_data['Percentage'] + std_data['Percentage'], 
                      alpha=0.2, label='Standard Deviation')
    all_data = []

    for file in filesDwave:
        df = pd.read_csv(file)
        df['group'] = df.index // 10
        true_counts = df.groupby('group')['Correct'].sum().reset_index()
        true_counts.columns = ['Nodes', 'True Count']
        true_counts["Nodes"] = true_counts["Nodes"] + 3
        true_counts["Percentage"] = (true_counts["True Count"] / 10)

        all_data.append(true_counts)
        #sns.lineplot(x='Nodes', y='Percentage', data=true_counts, label=df["Solver"][0])

    # Combine all data for mean and std calculation
    combined_data = pd.concat(all_data)
    mean_data = combined_data.groupby('Nodes')['Percentage'].mean().reset_index()
    std_data = combined_data.groupby('Nodes')['Percentage'].std().reset_index()

    # Plotting mean and standard deviation
    sns.lineplot(x='Nodes', y='Percentage', data=mean_data, label="D-Wave QPU")

    plt.fill_between(mean_data['Nodes'], 
                     mean_data['Percentage'] - std_data['Percentage'], 
                     mean_data['Percentage'] + std_data['Percentage'], 
                      alpha=0.2, label='Standard Deviation')

    plt.title('Success Rate by QUBO Problem Size Across Multiple Experiments')
    plt.xlabel('Node Count')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig("mean_accuracy.svg", format='svg')
    plt.show()


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

def plot_experiment_time_std(files,save_as_svg):
    num_subplots = len(files)
    scaler = MinMaxScaler()

    # Setting up the figure for subplots in a row
    fig, axs = plt.subplots(1, num_subplots, figsize=(5 * num_subplots, 4))
    plt.style.use(['science', 'ieee'])
    sns.set_palette("tab10")

    def string_to_dict(s):
        pairs = s.split(" ")
        dict_result = {}
        for pair in pairs:
            key, value = pair.split("=")
            if value.replace('.', '', 1).isdigit():
                value = float(value)
            elif value == "None":
                value = None
            dict_result[key] = value
        return dict_result

    # Process and plot each group of files
    for i, group in enumerate(files):
        all_data = []

        for file in group:
            df = pd.read_csv(file)
            df["Solve Time"] = df["Solve Time"].apply(string_to_dict)
            df["Total Solve Time"] = df["Solve Time"].apply(lambda x: x.get('total', None))
            df['group'] = df.index // 50
            mean_time = df.groupby('group')['Total Solve Time'].mean().reset_index()
            mean_time.columns = ['Interval', 'Mean Solve Time']
            mean_time["Interval"] = mean_time["Interval"] + 3
            all_data.append(mean_time)

        combined_data = pd.concat(all_data)
        mean_by_interval = combined_data.groupby('Interval')['Mean Solve Time'].mean().reset_index()
        mean_by_interval.columns = ['Interval', 'Mean Solve Time']
        std_data = combined_data.groupby('Interval')['Mean Solve Time'].std().reset_index()

        print(mean_by_interval)
        print(std_data)
        # Plotting on the respective subplot
        ax = axs[i] if num_subplots > 1 else axs
        sns.lineplot(x='Interval', y='Mean Solve Time', data=mean_by_interval, ax=ax, label=f'{df["Solver"][0]}')
        ax.set_title(f'Mean Solve Time by Node Count {df["Solver"][0]}')
        ax.set_xlabel('Node Count')
        ax.set_ylabel('Mean Solve Time')
        ax.fill_between(mean_by_interval['Interval'], 
                     mean_by_interval['Mean Solve Time'] - std_data['Mean Solve Time'], 
                     mean_by_interval['Mean Solve Time'] + std_data['Mean Solve Time'], 
                      alpha=0.2, label='Standard Deviation')
        ax.legend()
       

    plt.tight_layout()
    if save_as_svg:

        plt.savefig("mean_solve_time_SA-SAGA.svg", format='svg')
    plt.show()

In [12]:


def plot_qubit_scaling():
    edge_based = "47bc2ad2-935b-4198-8d14-d58f41994684.csv"
    node_based ="87cdccbc-0d97-4838-adf4-83e989db8b58.csv"
    node_based_3 ="c92adf00-4e02-4cd7-b592-0de56d2db837.csv"
    plt.style.use(['science', 'ieee'])
    sns.set_palette("tab10")
    plt.figure(figsize=(8, 4))

    df = pd.read_csv(edge_based)
    df['Run Number'] = df['Run Number'] + 3

    # Set x-ticks to unique values of 'Run Number'
    x_ticks = df['Run Number'].unique()
    plt.xticks(x_ticks)

    sns.lineplot(x='Run Number', y='Dimensions', data=df,label="Edge_Based")
    
    df = pd.read_csv(node_based)
    df['Run Number'] = df['Run Number'] + 3

    # Set x-ticks to unique values of 'Run Number'
    x_ticks = df['Run Number'].unique()
    plt.xticks(x_ticks)

    sns.lineplot(x='Run Number', y='Dimensions', data=df,label="Node_Based_max")

    df = pd.read_csv(node_based_3)
    df['Run Number'] = df['Run Number'] + 3

    # Set x-ticks to unique values of 'Run Number'
    x_ticks = df['Run Number'].unique()
    plt.xticks(x_ticks)

    sns.lineplot(x='Run Number', y='Dimensions', data=df,label="Node_Based_3")

    plt.title('QUBO Dimension in respect to Node count')
    plt.xlabel('Node Count')
    plt.ylabel('QUBO Dimension')
    plt.legend()
    plt.savefig("QUBO_dimensions.svg", format='svg')


    
    

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scienceplots

def plot_experiment_std_qbsolv(filesQBsolv):
    plt.figure(figsize=(8, 4))
    plt.style.use(['science','ieee'])
    sns.set_palette("tab10")

    all_data = []

    for file in filesQBsolv:
        df = pd.read_csv(file)
        df['group'] = df.index // 6
        true_counts = df.groupby('group')['Correct'].sum().reset_index()
        true_counts.columns = ['Nodes', 'True Count']
        true_counts["Nodes"] = true_counts["Nodes"] + 3
        true_counts["Percentage"] = (true_counts["True Count"] / 6)

        all_data.append(true_counts)
        #sns.lineplot(x='Nodes', y='Percentage', data=true_counts, label=df["Solver"][0])

    # Combine all data for mean and std calculation
    combined_data = pd.concat(all_data)
    mean_data = combined_data.groupby('Nodes')['Percentage'].mean().reset_index()
    std_data = combined_data.groupby('Nodes')['Percentage'].std().reset_index()

    # Plotting mean and standard deviation
    sns.lineplot(x='Nodes', y='Percentage', data=mean_data, label="D-Wave QBSolve")

    plt.fill_between(mean_data['Nodes'], 
                     mean_data['Percentage'] - std_data['Percentage'], 
                     mean_data['Percentage'] + std_data['Percentage'], 
                      alpha=0.2, label='Standard Deviation')
   

    plt.title('Success Rate by QUBO Problem Size Across Multiple Experiments')
    plt.xlabel('Node Count')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig("mean_accuracy_qbsolve.svg", format='svg')
    plt.show()


In [14]:
import networkx as nx
import matplotlib.pyplot as plt
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute,transpile

from numpy import pi
from qiskit.circuit.library import QFT
from qiskit.visualization import *
from qiskit.circuit.library import MCXGate

from qiskit.circuit.library import RGQFTMultiplier

In [15]:
G = nx.DiGraph()
G.add_edge('Node1', 'Node2', weight=2.0)
G.add_edge('Node1', 'Node3', weight=1.0)
G.add_edge('Node2', 'Node1', weight=1.0)
G.add_edge('Node2', 'Node3', weight=2.0)
G.add_edge('Node3', 'Node1', weight=2.0)
G.add_edge('Node3', 'Node2', weight=1.0)


In [ ]:
def preprocessing(graph):
    edges = graph.edges(data = True)

    #getting the max number of decimal places to be factor x in 10^x
    max_decimal = max([len(str(x[2]["weight"]).split(".")[1]) if str(x[2]["weight"]).split(".")[1] != "0" else 0 for x in edges ])
    print(max_decimal)

    for e in edges:
        e[2]["weight"] = e[2]["weight"] * 10**max_decimal
#    edges = [e[2]["weight"]*10**max_decimal for e in edges]

    #we return our max_decimal value to use in our bit flip as our goul is to find arbitrage we need to flip combinations that have a result
    #bigger than 1 * 10^max_decimal as they would also yield a gain in floating point multiplicaiton
    return max_decimal 


#perform n times addition to archive multiplication 
def add(qc):   
    for i in range(qc.qregs[1].size):
      qc.barrier()
      for j in range(qc.qregs[1].size-i):
        qc.cp((2*pi)/2**(j+1), qc.qregs[1][i],qc.qregs[2][j+i])


def create_circuit(graph, max_decimal:int, cycle_lenght:int):
    
    max_weight = max([x[2]["weight"] for x in graph.edges(data=True)])
    if max_decimal != 0:
        size_add_register=len(format((10**max_decimal)**cycle_lenght,"b"))
    else:
        size_add_register=len(format(int(max_weight)**cycle_lenght,"b"))

    num_nodes = graph.number_of_nodes()
    
    edges = QuantumRegister(graph.number_of_edges())
    
    add = QuantumRegister(4)#size_add_register)
    result = QuantumRegister(4)#size_add_register)
    ancilla =QuantumRegister(2)
    ancilla_2 = QuantumRegister(1)
    flip = QuantumRegister(1)
    c = ClassicalRegister(graph.number_of_edges())
    qc = QuantumCircuit(edges,add,result,ancilla,ancilla_2,flip,c)
    qc.h(edges)
    qc.x(flip)
    qc.h(flip)
    qc.barrier()
    return qc


def create_edge_reg(qc,graph):
    num_edges = graph.number_of_edges()
    edges = list(graph.edges())
    print(num_edges)
    edge_reg = {edges[i]:qc.qregs[0][i] for i in range(num_edges)}
    return edge_reg


def constraint_1(qc, graph, edge_reg):
    #as first constraint we enforce that for each node we can use a max of one incomeing and one outgoing edge
    qc.barrier()
    qc.x(qc.qregs[3])
    qc.x(qc.qregs[2])
         
    qc.barrier()

    for i, edge_1 in  enumerate(graph.edges()):
        for edge_2 in list(graph.edges())[i:]:
            if edge_1 == edge_2:
                continue
            if edge_1[0] == edge_2[0] and i<4:
                
                qc.mcx(target_qubit=qc.qregs[2][int(edge_1[0][-1])-1] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                
                """qc.x(edge_reg[edge_2])
                qc.mcx(target_qubit=qc.qregs[3][int(edge_1[0][-1])-1] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                qc.x(edge_reg[edge_1])
                qc.x(edge_reg[edge_2])"""
                qc.barrier()
                
            elif edge_1[0] == edge_2[0]:
                new_i = i-4
                
                qc.mcx(target_qubit=qc.qregs[3][new_i] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                
                """qc.x(edge_reg[edge_2])
                qc.mcx(target_qubit=qc.qregs[3][int(edge_1[0][-1])-1] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                qc.x(edge_reg[edge_1])
                qc.x(edge_reg[edge_2])"""
                qc.barrier()

            #ingoing edges
            if edge_1[1] == edge_2[1] and i <2:
                qc.mcx(target_qubit=qc.qregs[2][int(edge_1[1][-1])] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                
                """qc.x(edge_reg[edge_2])
                qc.mcx(target_qubit=qc.qregs[3][int(edge_1[1][-1])+2] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                qc.x(edge_reg[edge_1])
                qc.x(edge_reg[edge_2])"""
                qc.barrier()
                
            elif edge_1[1] == edge_2[1]:
                new_i = i-3

                qc.mcx(target_qubit=qc.qregs[3][new_i] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                
                """qc.x(edge_reg[edge_2])
                qc.mcx(target_qubit=qc.qregs[3][int(edge_1[1][-1])+2] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                qc.x(edge_reg[edge_1])
                qc.x(edge_reg[edge_2])"""
                qc.barrier()
    control_qubits = qc.qregs[3][:] + qc.qregs[2][:]  # All qubits from the 5th and 3rd registers
    qc.mcx(target_qubit=qc.qregs[1][0], control_qubits= control_qubits)


def check_tuple(arr, x, y):
    return any(t == (x, y) or t == (y, x) for t in arr)

def constraint_2(qc,graph,edge_reg):
    qc.barrier()
    qc.barrier()
    qc.barrier()


    qc.cx(qc.qregs[0][0],qc.qregs[3][0])
    qc.cx(qc.qregs[0][1],qc.qregs[3][0])
    qc.cx(qc.qregs[0][2],qc.qregs[3][1])
    qc.cx(qc.qregs[0][4],qc.qregs[3][1])
    qc.ccx(qc.qregs[3][0],qc.qregs[3][1],target_qubit= qc.qregs[1][1])
    qc.cx(qc.qregs[0][0],qc.qregs[3][0])
    qc.cx(qc.qregs[0][1],qc.qregs[3][0])
    qc.cx(qc.qregs[0][2],qc.qregs[3][1])
    qc.cx(qc.qregs[0][4],qc.qregs[3][1])


    
def constraint_1_r(qc, graph, edge_reg):
    #as first constraint we enforce that for each node we can use a max of one incomeing and one outgoing edge
    
         
    qc.barrier()

    for i, edge_1 in  enumerate(graph.edges()):
        for edge_2 in list(graph.edges())[i:]:
            if edge_1 == edge_2:
                continue
            if edge_1[0] == edge_2[0] and i<4:
                
                qc.mcx(target_qubit=qc.qregs[2][int(edge_1[0][-1])-1] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                
                """qc.x(edge_reg[edge_2])
                qc.mcx(target_qubit=qc.qregs[3][int(edge_1[0][-1])-1] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                qc.x(edge_reg[edge_1])
                qc.x(edge_reg[edge_2])"""
                qc.barrier()
                
            elif edge_1[0] == edge_2[0]:
                new_i = i-4
                
                qc.mcx(target_qubit=qc.qregs[3][new_i] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                
                """qc.x(edge_reg[edge_2])
                qc.mcx(target_qubit=qc.qregs[3][int(edge_1[0][-1])-1] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                qc.x(edge_reg[edge_1])
                qc.x(edge_reg[edge_2])"""
                qc.barrier()

            if edge_1[1] == edge_2[1] and i <2:
                print(i)
                qc.mcx(target_qubit=qc.qregs[2][int(edge_1[1][-1])] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                
                """qc.x(edge_reg[edge_2])
                qc.mcx(target_qubit=qc.qregs[3][int(edge_1[1][-1])+2] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                qc.x(edge_reg[edge_1])
                qc.x(edge_reg[edge_2])"""
                qc.barrier()
                
            elif edge_1[1] == edge_2[1]:
                new_i = i-3

                qc.mcx(target_qubit=qc.qregs[3][new_i] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                
                """qc.x(edge_reg[edge_2])
                qc.mcx(target_qubit=qc.qregs[3][int(edge_1[1][-1])+2] ,control_qubits=[edge_reg[edge_2],edge_reg[edge_1]])
                qc.x(edge_reg[edge_1])
                qc.x(edge_reg[edge_2])"""
                qc.barrier()
    qc.barrier()
    qc.x(qc.qregs[3])
    qc.x(qc.qregs[2])



def constraint_3(qc,graph, edge_reg):
    #responsible for the cycele length
    qft_invers =QFT(qc.qregs[1].size, inverse = True,do_swaps= False)
    qft = QFT(qc.qregs[1].size, do_swaps= False)
    result = qc.qregs[2]
    for i, edge_1 in  enumerate(graph.edges()):
        qc.barrier()

        qc.append(qft,result)
        
    
        for l in range(4):
          qc.cp((2*pi)/2**(l+1), qc.qregs[0][i],result[l])
        
                
                
        qc.append(qft_invers,result)
    qc.x(result[2])
    qc.x(result[3])
    qc.mcx(control_qubits=result,target_qubit= qc.qregs[1][2])

    #reverse
    qc.x(result[2])
    qc.x(result[3])
    for i, edge_1 in  enumerate(graph.edges()):
        qc.barrier()

        qc.append(qft,result)
        
    
        for l in range(4):
          qc.cp(-(2*pi)/2**(l+1), qc.qregs[0][i],result[l])
        
                
                
        qc.append(qft_invers,result)



def result_to_add(qc,edge):
    for i in range(qc.qregs[2].size):
        qc.mcx(target_qubit= qc.qregs[1][i], control_qubits=[qc.qregs[2][i], edge])



def multi(qc,G,edge_reg):
    edges = list(G.edges(data=True))
    qft_invers =QFT(qc.qregs[1].size, inverse = True,do_swaps= False)
    qft = QFT(qc.qregs[1].size, do_swaps= False)
    for j,edge in enumerate(edges):

        qubit = edge_reg[(edge[0],edge[1])]
        

        weight = int(edge[2]["weight"])
        e = (edge[0], edge[1])
        if weight != 1:
            qc.barrier()
            #qc.reset(qc.qregs[1])

            result_to_add(qc,qubit)
            
            qc.append(qft,qc.qregs[2])
            for i in range(weight-1):
                add(qc)
                
            qc.append(qft_invers,qc.qregs[2])
            qc.barrier()
            if j == 0:
                qc.ccx(qc.qregs[2][1],qc.qregs[0][0],qc.qregs[1][0])
            if j == 3:
                qc.ccx(qc.qregs[2][2],qc.qregs[0][3],qc.qregs[1][1])
            if j == 4:
                qc.ccx(qc.qregs[2][3],qc.qregs[0][4],qc.qregs[1][2])
              

        
        

def flip(qc):

    qc.mcx(target_qubit=qc.qregs[5], control_qubits=[qc.qregs[2][3]])
    
    qc.barrier()



def diffusor(qc):
    #TODO: change so this works for every graph
    qc.barrier()
    qc.h(qc.qregs[0])
    qc.x(qc.qregs[0])
    qc.h(qc.qregs[0][5])
    qc.mcx(target_qubit=qc.qregs[0][5],control_qubits =[qc.qregs[0][0],qc.qregs[0][1],qc.qregs[0][2],qc.qregs[0][3],qc.qregs[0][4]])
    qc.h(qc.qregs[0][5])
    qc.x(qc.qregs[0])
    qc.h(qc.qregs[0])
    qc.barrier()
    


def measure(qc):
    qc.measure(qc.qregs[0],qc.cregs[0])


max_decimal = preprocessing(G)
qc = create_circuit(G,max_decimal,3)
edge_reg = create_edge_reg(qc,G)
for i in range(7):
    constraint_3(qc,G,edge_reg)
    constraint_1(qc,G,edge_reg)
    constraint_1_r(qc,G,edge_reg)

    constraint_2(qc,G,edge_reg)
    qc.barrier()
    qc.mcx(control_qubits=[qc.qregs[1][0],qc.qregs[1][1],qc.qregs[1][2]],target_qubit= qc.qregs[4][0])

    constraint_3(qc,G,edge_reg)
    constraint_1(qc,G,edge_reg)
    constraint_1_r(qc,G,edge_reg)

    constraint_2(qc,G,edge_reg)
    qc.barrier()
    qc.cx(qc.qregs[4][0],qc.qregs[2][0])
    qc.cx(qc.qregs[2][0],qc.qregs[4][0])


    multi(qc,G,edge_reg)

    

    flip(qc)
    qc.mcx(target_qubit=qc.qregs[2][3], control_qubits=[qc.qregs[0][0],qc.qregs[0][3],qc.qregs[0][4]])

    

    
    diffusor(qc)


measure(qc)